# Needed imports and utility

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from comtypes.client import CreateObject

import ansys.speos.core as core
from ansys.speos.core.speos import Speos
import ansys.speos.workflow as workflow

tests_data_path = os.path.join(os.path.join(os.path.abspath(''), os.path.pardir), "assets")

def open_file(file):
    dpf_instance = None
    if file.endswith("xmp") or file.endswith("XMP"):
        dpf_instance = CreateObject("XMPViewer.Application")
        dpf_instance.OpenFile(file)
        res = dpf_instance.ExportXMPImage(file+".png", 1)
        if res:
            img = mpimg.imread(file+".png")
            plt.imshow(img)
            plt.axis("off")   # turns off axes
            plt.axis("tight")  # gets rid of white border
            plt.axis("image")  # square up the image instead of filling the "figure" space
            plt.show()
    elif file.endswith("hdr") or file.endswith("HDR"):
        dpf_instance = CreateObject("HDRIViewer.Application")
        dpf_instance.OpenFile(file)
        dpf_instance.Show(1)
    elif file.endswith("png") or file.endswith("PNG"):
        img = mpimg.imread(file)
        plt.imshow(img)
        plt.axis("off")   # turns off axes
        plt.axis("tight")  # gets rid of white border
        plt.axis("image")  # square up the image instead of filling the "figure" space
        plt.show()
    return dpf_instance


def print_message(msg):
    print(core.protobuf_message_to_str(msg))

# Create connected speos rpc server

In [ ]:
speos = Speos(host="localhost", port=50051)
workflow.clean_all_dbs(speos.client)

# Create SpeosSimulationUpdate

In [ ]:
sim_file = os.path.join(tests_data_path, "Env_Simplified_Empty.speos", "Env_Simplified_Empty.speos")
env_file = os.path.join(tests_data_path, "Env_Simplified.speos", "Env_Simplified.speos")
blue_car_file = os.path.join(tests_data_path, "BlueCar.speos", "BlueCar.speos")
red_car_file = os.path.join(tests_data_path, "RedCar.speos", "RedCar.speos")

my_scene_simu = workflow.SpeosSimulationUpdate(speos, sim_file, clean_dbs=False)
env_scene = workflow.SpeosSimulationUpdate(speos, env_file, clean_dbs=False)
blue_car_scene = workflow.SpeosSimulationUpdate(speos, blue_car_file, clean_dbs=False)
red_car_scene = workflow.SpeosSimulationUpdate(speos, red_car_file, clean_dbs=False)

# Assemble multiple scenes

In [ ]:
env_position = core.AxisSystem()
env_position.origin = [0.0, 0.0, 0.0]
env_position.x_vect = [1.0, 0.0, 0.0]
env_position.y_vect = [0.0, 1.0, 0.0]
env_position.z_vect = [0.0, 0.0, 1.0]

blue_car_position = core.AxisSystem()
blue_car_position.origin = [1984.87, 0, 34660.06]
blue_car_position.x_vect = [0.0, 0.0, 1.0]
blue_car_position.y_vect = [-1.0, 0.0, 0.0]
blue_car_position.z_vect = [0.0, 1.0, 0.0]

red_car_position = core.AxisSystem()
red_car_position.origin = [-3749, 0.0, 48000]
red_car_position.x_vect = [1.0, 0.0, 0.0]
red_car_position.y_vect = [0.0, 0.0, -1.0]
red_car_position.z_vect = [0.0, 1.0, 0.0]

my_scene_simu.add_scene(env_scene, env_position)
my_scene_simu.add_scene(red_car_scene, red_car_position)
my_scene_simu.add_scene(blue_car_scene, blue_car_position)

# Preview the scene

In [ ]:
my_scene_simu.preview()

# Add a camera sensor to the scene

In [ ]:
camera_input_path = os.path.join(tests_data_path, "CameraInputFiles")
camera_parameters = workflow.PhotometricCameraSensorParameters()
camera_parameters.name = "Camera_Sensor"
camera_parameters.transmittance_file = os.path.join(camera_input_path, "CameraTransmittance.spectrum")
camera_parameters.distortion_file = os.path.join(camera_input_path, "CameraDistortion_190deg.OPTDistortion")
camera_parameters.color_mode_red_spectrum_file = os.path.join(camera_input_path, "CameraSensitivityRed.spectrum")
camera_parameters.color_mode_green_spectrum_file = os.path.join(camera_input_path, "CameraSensitivityGreen.spectrum")
camera_parameters.color_mode_blue_spectrum_file = os.path.join(camera_input_path, "CameraSensitivityBlue.spectrum")
camera_parameters.wavelengths_start = 380
camera_parameters.wavelengths_end = 780
camera_parameters.wavelengths_sampling = 21
camera_parameters.focal_length = 7
camera_parameters.imager_distance = 1
camera_parameters.f_number = 20
camera_parameters.horizontal_pixel = 964
camera_parameters.vertical_pixel = 544
camera_parameters.width = 8.0976
camera_parameters.height = 4.5696

camera_props = workflow.CameraSensorProperties()
camera_props.origin = [-2118.67, 1500, 11000]
camera_props.x_vect = [-0.999694, 0.0, -0.0247255]
camera_props.y_vect = [0.0, 1.0, 0.0]
camera_props.z_vect = [0.0, 0.0, -1.0]

my_scene_simu.add_camera_sensor(camera_parameters, camera_props)

# Compute the first simulation

In [ ]:
job_link = my_scene_simu.compute("first_job")
print_message(job_link.get_results())

# Open and view the result file

In [ ]:
speos_result_file = job_link.get_results().results[4].path
open_file(speos_result_file)
speos_result_file = job_link.get_results().results[1].path
open_file(speos_result_file)

# Move the assembly components

In [ ]:
red_car_position_update = red_car_position
red_car_position_update.origin = [0.0, 0.0, 48000]
blue_car_position_update = blue_car_position
blue_car_position_update.origin = [1984.87, 0, 24660.06]

new_body_positions = {
    red_car_scene.scene.get().name: red_car_position_update,
    blue_car_scene.scene.get().name: blue_car_position_update,
}
my_scene_simu.update_scene_part_position(new_body_positions)

# Compute the modified simulation with GPU
Note: a student license does not support GPU compute

In [ ]:
job_link = my_scene_simu.compute("second_job", compute_type="gpu")
print_message(job_link.get_results())

# Open and view the modified result file

In [ ]:
speos_result_file = job_link.get_results().results[4].path
open_file(speos_result_file)
speos_result_file = job_link.get_results().results[1].path
open_file(speos_result_file)

# Move the camera position

In [ ]:
camera_props_update = workflow.CameraSensorProperties()
camera_props_update.origin = [-2118.67, 1500.0, 21000.0]
camera_props_update.x_vect = [-0.999694, 0.0, -0.0247255]
camera_props_update.y_vect = [0.0, 1.0, 0.0]
camera_props_update.z_vect = [0.0, 0.0, -1.0]

my_scene_simu.update_sensor(camera_parameters, camera_props_update)

# Compute the modified simulation
Note: a student license does not support GPU compute

In [ ]:
job_link = my_scene_simu.compute("third_job", compute_type="gpu")
print_message(job_link.get_results())

# Open and view the modified simulation result files

In [ ]:
speos_result_file = job_link.get_results().results[4].path
open_file(speos_result_file)
speos_result_file = job_link.get_results().results[1].path
open_file(speos_result_file)

# Clean after usage

In [ ]:
job_link.delete()
my_scene_simu.close()